In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()
# Country Data COUNTRY table
COUNTRY_df = pd.read_sql_query("select * from COUNTRY;", conn)
# League data from LEAGUE
LEAGUE_df = pd.read_sql_query("select * from LEAGUE;", conn)
# Match data from Matches - just pulling 1 year for now -2015/2016 might be latest year
# You can remove restriction if you want
MATCH_df = pd.read_sql_query("select * from MATCH where SEASON = '2015/2016';", conn)
# Match data from Matches - just pulling 1 and relevant data for now - we can remove restriction later if we want
MATCH_cutdown_df = pd.read_sql_query("SELECT SEASON, STAGE, DATE, MATCH_API_ID, HOME_TEAM_API_ID, AWAY_TEAM_API_ID, HOME_TEAM_GOAL, AWAY_TEAM_GOAL FROM MATCH WHERE SEASON = '2015/2016';",
conn)

In [2]:
MATCH_cutdown_df

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,2015/2016,1,2015-07-24 00:00:00,1979832,9997,8342,2,1
1,2015/2016,1,2015-07-25 00:00:00,1979833,8571,9985,2,1
2,2015/2016,1,2015-07-25 00:00:00,1979834,9987,1773,3,1
3,2015/2016,1,2015-07-25 00:00:00,1979835,8573,8203,3,1
4,2015/2016,1,2015-07-25 00:00:00,1979836,10000,9994,3,1
...,...,...,...,...,...,...,...,...
3321,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,0
3322,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,2
3323,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,0
3324,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,0
